# Import Packages

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

from luwiji.text_proc import illustration

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import Data

In [3]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract TFIDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

C:\Users\ASUS\miniconda3\envs\jcop\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\ASUS\miniconda3\envs\jcop\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


# TF-IDF Similarity => Document Similarity

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [9]:
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]], dtype=int64)

In [10]:
df.teks

0       Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1       Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2       Penyimpangan di Setpres Seolah Terjadi Sekaran...
3       Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK...
4       Stop Kekerasan, Elite agar Duduk Bersama\nSeju...
                              ...                        
2003    Tersangka Peledakan Granat di Bulungan Ditangk...
2004    Soal Operasi Pasar Murni Beras   Pedagang Yaki...
2005    Penjualan Indomobil Langgar Prosedur\n\nSelain...
2006    Belum Jelas Motif  Peledakan Granat di Blok M\...
2007    ANALISIS EKONOMI SJAHRIR    BBM dan PKPS: Menu...
Name: teks, Length: 2008, dtype: object

In [11]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [13]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [14]:
df.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

# Keyword Extraction

In [15]:
vocab = tfidf.get_feature_names_out()
vocab[-10:]

array(['zuniga memilih', 'zunnatul', 'zunnatul mafruhah', 'zurich',
       'zurich northholt', 'zw', 'zw suparman', 'zw tim', 'zx',
       'zx diserbu'], dtype=object)

In [17]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [19]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274212, 365469, 365468, ..., 386379, 436652, 169219], dtype=int64)

In [21]:
vocab[169219]

'ginandjar'

In [22]:
vocab[436652]

'putusan'

In [23]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [26]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names_out()
    
    sortef_idf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [27]:
extract_keywords_tfidf(df.teks[0], tfidf)

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']